## Saving and loading models, with application to the EuroSat dataset

Creating a neural network that classifies land uses and land covers from satellite imagery. We will save callbacks to save model and use it later. 



In [ ]:
#### PACKAGE IMPORTS ####

import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
import os
import numpy as np
import pandas as pd




#### The EuroSAT dataset
Using the [EuroSAT dataset](https://github.com/phelber/EuroSAT). It consists of 27000 labelled Sentinel-2 satellite images of different land uses: residential, industrial, highway, river, forest, pasture, herbaceous vegetation, annual crop, permanent crop and sea/lake.

In [ ]:
###https://raw.githubusercontent.com/phelber/EuroSAT/master/eurosat_overview_small.jpg

#### Import the data

The dataset you will train your model on is a subset of the total data, with 4000 training images and 1000 testing images, with roughly equal numbers of each class. The code to import the data is provided below.

In [ ]:

def load_eurosat_data():
    data_dir = 'data/'
    x_train = np.load(os.path.join(data_dir, 'x_train.npy'))
    y_train = np.load(os.path.join(data_dir, 'y_train.npy'))
    x_test  = np.load(os.path.join(data_dir, 'x_test.npy'))
    y_test  = np.load(os.path.join(data_dir, 'y_test.npy'))
    return (x_train, y_train), (x_test, y_test)

(x_train, y_train), (x_test, y_test) = load_eurosat_data()
x_train = x_train / 255.0
x_test = x_test / 255.0

#### Building the neural network model

In [ ]:

def get_new_model(input_shape):
    model = Sequential([
        Conv2D(filters=16, input_shape=input_shape, kernel_size=(3, 3), 
               activation='relu',padding='SAME' ,name='conv_1'),
        Conv2D(filters=8, kernel_size=(3, 3), activation='relu',padding='SAME',name='conv_2'),
        MaxPooling2D(pool_size=(8, 8), name='pool_1'),
        Flatten(name='flatten'),
        Dense(units=32, activation='relu', name='dense_1'),
        Dense(units=10, activation='softmax', name='dense_2')
    ])
    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    return model
    

    

In [ ]:
x_train[0].shape

#### Compile and evaluate the model

In [ ]:

model = get_new_model(x_train[0].shape)

In [ ]:
# Run this cell to define a function to evaluate a model's test accuracy

def get_test_accuracy(model, x_test, y_test):
    """Test model classification accuracy"""
    test_loss, test_acc = model.evaluate(x=x_test, y=y_test, verbose=0)
    print('accuracy: {acc:0.3f}'.format(acc=test_acc))

In [ ]:
# Print the model summary and calculate its initialised test accuracy

model.summary()
get_test_accuracy(model, x_test, y_test)


We will create three callbacks:
- `checkpoint_every_epoch`: checkpoint that saves the model weights every epoch during training
- `checkpoint_best_only`: checkpoint that saves only the weights with the highest validation accuracy. Use the testing data as the validation data.
- `early_stopping`: early stopping object that ends training if the validation accuracy has not improved in 3 epochs.

In [ ]:

def get_checkpoint_every_epoch():
    checkpoint_every_epoch='checkpoints_every_epoch/checkpoint_{epoch:03d}'
    checkpoint_epoch=ModelCheckpoint(filepath=checkpoint_every_epoch,frequency='epoch',save_weights_only=True,verbose=1,save_freq='epoch',monitor='val_accuracy')
    return checkpoint_epoch
    """
    This function returns a ModelCheckpoint object that:
    - saves the weights only at the end of every epoch
    - saves into a directory called 'checkpoints_every_epoch' inside the current working directory
    """
    
    


def get_checkpoint_best_only():
    checkpoint_best_path='checkpoints_best_only/checkpoint'
    checkpoint_best=ModelCheckpoint(filepath=checkpoint_best_path,frequency='epoch',save_weights_only=True,verbose=1,save_freq='epoch',save_best_only=True,monitor='val_accuracy')
    return checkpoint_best
    
    """
    This function returns a ModelCheckpoint object that:
    - saves only the weights that generate the highest validation (testing) accuracy
    - saves into a directory called 'checkpoints_best_only' inside the current working directory
    """
    
    

In [ ]:

def get_early_stopping():
    early_stopping=tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',patience=3
)
    return early_stopping
    
    """
    This function returns an EarlyStopping callback that stops training when
    the validation (testing) accuracy has not improved in the last 3 epochs.
    """
    
    

In [ ]:
checkpoint_every_epoch = get_checkpoint_every_epoch()
checkpoint_best_only = get_checkpoint_best_only()
early_stopping = get_early_stopping()

#### Training model using the callbacks


In [ ]:
callbacks = [checkpoint_every_epoch, checkpoint_best_only, early_stopping]
model.fit(x_train, y_train, epochs=50, validation_data=(x_test, y_test), callbacks=callbacks)

In [ ]:
!ls -lh checkpoints_every_epoch

In [ ]:
!ls -lh checkpoints_best_only

In [ ]:
print(tf.train.latest_checkpoint("checkpoint_every_epoch"))

#### Create new instance of model and load on both sets of weights

In [ ]:

def get_model_last_epoch(model):
    model.load_weights(tf.train.latest_checkpoint("checkpoints_every_epoch"))
    return model
    
def get_model_best_epoch(model):
    model.load_weights('checkpoints_best_only/checkpoint')
    return model


In [ ]:
model_last_epoch = get_model_last_epoch(get_new_model(x_train[0].shape))
model_best_epoch = get_model_best_epoch(get_new_model(x_train[0].shape))
print('Model with last epoch weights:')
get_test_accuracy(model_last_epoch, x_test, y_test)
print('Model with best epoch weights:')
get_test_accuracy(model_best_epoch, x_test, y_test)